In [1]:
import numpy as np
import scipy
import pyscf
import pyscf.tools

import orbitalpartitioning

In [2]:
molecule = """
H 0.0 0.0 0.0
H 2.0 0.0 0.0
H 4.0 0.0 0.0
He 0.0 2.0 0.0
He 2.0 2.0 0.0
He 4.0 2.0 0.0
He 6.0 2.0 0.0
He 8.0 2.0 0.0
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   3, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

symmetry:  Cs


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 5  num. singly occ = 3
init E= -15.3068786812968
HOMO (A') = -0.0514772431110082  LUMO (A') = 0.697658707042951
cycle= 1 E= -15.7089655763235  delta_E= -0.402  |g|= 0.13  |ddm|= 1.16
HOMO (A') = -0.120379087595762  LUMO (A') = 0.681816406674529
cycle= 2 E= -15.7314732931088  delta_E= -0.0225  |g|= 0.0265  |ddm|= 0.388
HOMO (A') = -0.0883523449275507  LUMO (A') = 0.683457077168596
cycle= 3 E= -15.7323644181807  delta_E= -0.000891  |g|= 0.00231  |ddm|= 0.0787
HOMO (A') = -0.0848665358224598  LUMO (A') = 0.682733582208056
cycle= 4 E= -15.7323701283

# Define Fragments by AOs

In [3]:
# Find AO's corresponding to atoms 
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] in (0, 1, 2, 3):
        if ao[2] in ("1s",):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] in (4, 5):
        if ao[2] in ("1s", ):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 6:
        if ao[2] in ("1s", ):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)


[[0, 5, 10, 15], [20, 25], [30]]


# Define Projectors
We can choose to project onto the non-orthogonal AOs, or onto the symmetrically orthogonalized AOs.

In [4]:
# Define projectors
X = np.eye(nbas) 
X = scipy.linalg.sqrtm(S)
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])


# Project MOs onto all fragments
For each orbital block (Docc, Sing, Virt), project each subspace onto the full list of fragment AOs. This will determine our full active space.

In [6]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = orbitalpartitioning.spade_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact,Vact))

 Partition   40 orbitals into a total of    7 orbitals
            Index   Sing. Val. Space       
                0   0.91194820            1*
                1   0.84299860            0*
                2   0.83375253            0*
                3   0.82230440            0*
                4   0.82173939            1*
                5   0.81263553            0*
                6   0.73671137            1*
                7   0.17037483            2
                8   0.16235909            2
                9   0.15903726            2
               10   0.15338327            2
               11   0.14107874            2
               12   0.11205413            2
               13   0.05345282            2
               14   0.00034498            0


# Split active space into fragments

In [7]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1
for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = orbitalpartitioning.spade_partitioning((Cdocc, Csing, Cvirt), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = orbitalpartitioning.sym_ortho(Cfrags, S)

Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace: ", init_fspace)
print(" clusters   : ", clusters)



 Fragment:  [0, 5, 10, 15]


NameError: name 'orbitalpartioning' is not defined

# Make Integrals

In [ ]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)

(40, 1)
(40, 7)
(40, 40)


In [ ]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;

In [ ]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [ ]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [ ]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

In [ ]:
import numpy as np
Ccmf = np.load("Ccmf.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf.molden", Ccmf)
